In [1]:
from requests import Session
from tabulate import tabulate
import json
import io
# encoding=utf8

session = Session()
tokenResponse = session.post('https://api.etilbudsavis.dk/v2/sessions?api_key=00izwus560vpbdasf0o92m2fhaim4ynr')
token = (json.loads(tokenResponse.text))["token"]
if not token:
    print "Couldn't get token"
defaultHeaders = {'X-token': token, 'Origin': "https://etilbudsavis.dk/"}

In [2]:
#List Store. 
listResponse = session.get('https://api.etilbudsavis.dk/v2/stores?r_lat=59.95&r_lng=10.75&r_radius=700000&api_av=0.1.33&limit=100',
                              headers=defaultHeaders)
stores = json.loads(listResponse.text)
storeList = [[store['branding']['name'], store['id']] for store in stores]
print "-----------------List 100 stores-----------------------------"
print tabulate(storeList, headers=['All Stores', 'Id'])
#Should use the branding name instead of store id to filter
#selectedId = ['7edbLOE', '90f2F7y', 'd8dewIy', '2665CXz', '82a9zXz', 'b88bQ7y', '618fq21', '98c86Iq']
#selectedStoreList = [[store['branding']['name'], store['id']] for store in stores if (store['id'] in selectedId)]
#print tabulate(selectedStoreList, headers=['Selected Stores', 'Id'])

#Note: input names in lower-case
selectedBrandNames = ['coop mega', 'coop extra', 'coop marked', 'kiwi', 'meny', 'rema', 'matkroken', 
                      'coop obs', 'coop prix','joker', 'bunnpris', 'extra', 'spar']

-----------------List 100 stores-----------------------------
All Stores             Id
---------------------  -------
Matkroken Tåsen        7edbLOE
Apotek 1               d146811
SPAR                   2a7dV81
Norli                  ec06Mm3
Christiania Belysning  356ePLq
KIWI                   90f2F7y
MENY                   d8dewIy
Coop Mega Nydalen      2665CXz
Apotek 1               77f6011
Vitus Apotek           57e1mVq
Apotek 1               49b2Gy1
BilXtra                36346GD
Jernia                 84d5SND
Sport1                 188aLDq
Barnas Hus             acefKMD
G-Sport | G-MAX        5a29G48
Tilbords               bcd4GjA
Life Helsekost         4d08O6D
Vita                   ef73ozq
Coop Mega Ullevål      82a9zXz
KIWI                   b88bQ7y
Elkjøp                 de03l41
Närbutikken            67f3Wvq
Joker                  618fq21
REMA 1000              6d3dpaz
REMA 1000              793fTYz
Joker                  00f0A21
Plantasjen             36feM5q
Expert       

In [3]:
#List Catalog
#The store_ids filter of the API does not seem to work, so we have to filter ourself!
#storeIdStr = ','.join([selectedStore[1] for selectedStore in selectedStoreList])
catalogsResponse = session.get('https://api.etilbudsavis.dk/v2/catalogs?r_lat=59.95'+
                               '&r_lng=10.75&r_radius=700000&api_av=0.1.33&limit=100',
                               headers=defaultHeaders)
catalogs = json.loads(catalogsResponse.text)
catalogList = [[catalog['branding']['name'], catalog['store_id'], catalog['id']] for catalog in catalogs]
print tabulate(catalogList, headers=['All Catalogs', 'Store Id', 'Id'])

#Check if any the brand name contains any selectedBrandName
selectedCatalogs = [catalog for catalog in catalogs 
                    if len([brandName for brandName in selectedBrandNames if brandName in catalog['branding']['name'].lower()]) > 0]
selectedCatalogList = [[catalog['branding']['name'], catalog['id'], catalog['run_from'], 
                        next(brandName for brandName in selectedBrandNames if brandName in catalog['branding']['name'].lower())] 
                       for catalog in selectedCatalogs]
print "\n---------------------------------"
print tabulate(selectedCatalogList, headers=['Selected Catalogs', 'Id', 'Run From', 'Brand'])

All Catalogs               Store Id    Id
-------------------------  ----------  -------
Europris                   1a49Wk3     7ebekeq
JYSK                       52e71L3     5dd6bbq
Clas Ohlson                b7e84C1     09bdS4q
Expert                     6d78T91     9cedHeq
G-Sport | G-MAX            5a29G48     4defyeq
Elkjøp                     de03l41     e211RJq
Skeidar                    4e7fMkD     05bdz5q
Joker                      618fq21     3b83t9q
Nille                      0659dF3     3497UJq
Rusta                      2d99HSq     1f0cTJq
Jula                       12d0Ta3     89c2hJq
Vita                       ef73ozq     74afW9q
Jernia                     84d5SND     1742ENq
Staples                    3678GK1     2022zeq
SPAR                       2a7dV81     3a9emNq
Extra Bekkestua            98c86Iq     719819q
lefdal                     3fc34cy     6a9dSJq
G-Sport | G-MAX            5a29G48     7ab71eq
Enklere Liv                ce46Mdz     8430T4q
Møbelringen       

In [4]:
#Get offers in selected catalog
for catalogStore, catalogId, catalogRunFrom, catalogBrand in selectedCatalogList:
    print "--------------------------------------------"
    print "Requesting catalog "+catalogStore+" with id "+catalogId+"..."
    offersResponse = session.get('https://api.etilbudsavis.dk/v2/catalogs/'+catalogId+'/hotspots?r_lat=59.95&r_lng=10.75&r_radius=700000&api_av=0.1.33',
                        headers=defaultHeaders)
    offers = json.loads(offersResponse.text)
    print "Number of offers: "+ str(len(offers))
    if len(offers)==0:
        continue
    offerIdList = [offer['id'] for offer in offers]
    print "Parsing offers..."
    JsonFileName = catalogBrand + '_'+catalogId + "_" + catalogRunFrom + ".json"
    offerDescList = []
    for offerId in offerIdList:
        offerDescResponse = session.get('https://api.etilbudsavis.dk/v2/offers/'+offerId+'?r_lat=59.95&r_lng=10.75&r_radius=700000&api_av=0.1.33',
                                    headers=defaultHeaders)
        offerDescList.append(json.loads(offerDescResponse.text.encode('utf-8')))
    with io.open(JsonFileName, 'w', encoding="utf-8") as json_file:
        data = json.dumps(offerDescList, ensure_ascii=False)     
        json_file.write(unicode(data))
    print "Done."

--------------------------------------------
Requesting catalog Joker with id 3b83t9q...
Number of offers: 33
Parsing offers...
Done.
--------------------------------------------
Requesting catalog Extra Bekkestua with id 719819q...
Number of offers: 75
Parsing offers...
Done.
--------------------------------------------
Requesting catalog Coop Prix Hausmannsgate with id 8488s9q...
Number of offers: 50
Parsing offers...
Done.
--------------------------------------------
Requesting catalog Bunnpris Nydalen with id ea96pdq...
Number of offers: 0
--------------------------------------------
Requesting catalog Coop Mega Bekkestua with id 1a73n9q...
Number of offers: 37
Parsing offers...
Done.
--------------------------------------------
Requesting catalog Bunnpris Nydalen with id a7d3oNq...
Number of offers: 0
--------------------------------------------
Requesting catalog Coop Marked with id cbd0M9q...
Number of offers: 28
Parsing offers...
Done.
------------------------------------------

## Download deals from before

In [14]:
selectedBrandNames = ['coop mega', 'coop extra', 'coop marked', 'kiwi', 'meny', 'rema', 'matkroken', 
                      'coop obs', 'coop prix','joker', 'bunnpris', 'extra', 'spar']

catalogsResponse = session.get('https://api.etilbudsavis.dk/v2/catalogs?r_lat=59.95'+
                               '&r_lng=10.75&r_radius=700000&api_av=0.1.33'+
                               '&offset=10&limit=100'+
                               '&run_from=2015-01-30T23:00:00+0000'+
                               '&run_till=2015-02-13T22:59:59+0000',
                               headers=defaultHeaders)
catalogs = json.loads(catalogsResponse.text)
catalogList = [[catalog['branding']['name'], catalog['store_id'], catalog['id']] for catalog in catalogs]
print tabulate(catalogList, headers=['All Catalogs', 'Store Id', 'Id'])

#Check if any brand name contains any selectedBrandName
selectedCatalogs = [catalog for catalog in catalogs 
                    if len([brandName for brandName in selectedBrandNames if brandName in catalog['branding']['name'].lower()]) > 0]
selectedCatalogList = [[catalog['branding']['name'], catalog['id'], catalog['run_from'], 
                        next(brandName for brandName in selectedBrandNames if brandName in catalog['branding']['name'].lower())] 
                       for catalog in selectedCatalogs]
print "\n---------------------------------"
print tabulate(selectedCatalogList, headers=['Selected Catalogs', 'Id', 'Run From', 'Brand'])

All Catalogs               Store Id    Id
-------------------------  ----------  -------
Jula                       12d0Ta3     89c2hJq
Vita                       ef73ozq     74afW9q
Jernia                     84d5SND     1742ENq
Staples                    3678GK1     2022zeq
SPAR                       2a7dV81     3a9emNq
Extra Bekkestua            98c86Iq     719819q
lefdal                     3fc34cy     6a9dSJq
G-Sport | G-MAX            5a29G48     7ab71eq
Enklere Liv                ce46Mdz     8430T4q
Møbelringen                d2bdklD     7ecd54q
Coop Byggmix               6ed2RD3     4ec4Seq
Christiania Belysning      356ePLq     a2dcI4q
Coop Prix Hausmannsgate    6a76Ahz     8488s9q
Biltema                    d026SA1     d35c32q
Bohus                      0f62X4A     39afSVq
Bunnpris Nydalen           f00d501     ea96pdq
Coop Mega Bekkestua        045ekR1     1a73n9q
Montér                     acc2IOD     0467hNq
Obs! Alnabru               3a66jSA     a7c4u9q
Bunnpris Nydalen  

In [15]:
catalogsResponse

<Response [200]>

In [16]:
catalogs = json.loads(catalogsResponse.text)
